# DMTA Cross-Domain Consistency: Frequency vs Relaxation

The same discrete relaxation spectrum H(tau) should describe both E*(omega) and E(t). This notebook validates that Prony series fitted in one domain correctly predict the other.

## Learning Objectives
- Fit GMM to frequency-domain E*(omega) and time-domain E(t) independently
- Cross-predict: frequency Prony -> E(t) and relaxation Prony -> E*(omega)
- Compare relaxation spectra from both domains
- Understand H(tau) as a domain-independent material property

**Data**: pyvisco project (NREL, MIT License)

**Estimated Time:** 5-8 minutes (FAST_MODE), 10-15 minutes (full)

In [1]:
import gc
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

if os.path.abspath(os.path.join(os.getcwd(), '../..')) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()

np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

FAST_MODE = os.environ.get('FAST_MODE', '1') == '1'
print(f'FAST_MODE: {FAST_MODE}')

FAST_MODE: True


## 1. Load Both Domains

We load the frequency-domain master curve E*(omega) and the time-domain relaxation master curve E(t). Both are TTS-expanded master curves at the same reference temperature.

In [2]:
# Load data
data_dir = os.path.join(os.path.dirname(os.path.abspath('.')), 'dmta', 'data')
if not os.path.exists(data_dir):
    data_dir = os.path.join('.', 'data')

# Frequency domain
df_freq = pd.read_csv(os.path.join(data_dir, 'freq_user_master.csv'), skiprows=[1])
omega = 2 * np.pi * df_freq['f'].values  # Hz -> rad/s
E_stor = df_freq['E_stor'].values * 1e6  # MPa -> Pa
E_loss = df_freq['E_loss'].values * 1e6
E_star = E_stor + 1j * E_loss

# Time domain
df_time = pd.read_csv(os.path.join(data_dir, 'time_user_master.csv'), skiprows=[1])
df_time.columns = df_time.columns.str.strip()
t = df_time['t'].values
E_t = df_time['E_relax'].values * 1e6  # MPa -> Pa

print(f'Frequency domain: {len(omega)} pts, {np.log10(omega.max()/omega.min()):.1f} decades')
print(f'Time domain: {len(t)} pts, {np.log10(t.max()/t.min()):.1f} decades')
print(f"E' range: {E_stor.min()/1e6:.0f} - {E_stor.max()/1e6:.0f} MPa")
print(f'E(t) range: {E_t.min()/1e6:.0f} - {E_t.max()/1e6:.0f} MPa')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.loglog(omega, E_stor, 'ro', ms=3, alpha=0.5, label="E'")
ax1.loglog(omega, E_loss, 'bs', ms=3, alpha=0.5, label='E"')
ax1.set_xlabel(chr(969) + ' (rad/s)')
ax1.set_ylabel('Modulus (Pa)')
ax1.set_title('Frequency Domain: E*(omega)')
ax1.legend()

ax2.loglog(t, E_t, 'g-', lw=2)
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('E(t) (Pa)')
ax2.set_title('Time Domain: E(t)')

plt.tight_layout()
plt.close('all')

Frequency domain: 206 pts, 26.0 decades
Time domain: 481 pts, 30.7 decades
E' range: 89 - 9581 MPa
E(t) range: 86 - 1714 MPa


## 2. Fit GMM to Frequency Domain

In [3]:
from rheojax.models.multimode.generalized_maxwell import GeneralizedMaxwell

n_modes = 5 if FAST_MODE else 10

# Fit to E*(omega)
gmm_freq = GeneralizedMaxwell(n_modes=n_modes, modulus_type='tensile')

gmm_freq.fit(omega, E_star, test_mode='oscillation', optimization_factor=None)

E_freq_pred = gmm_freq.predict(omega, test_mode='oscillation')
E_freq_prime = E_freq_pred[:, 0]
E_freq_double = E_freq_pred[:, 1]

ss_res = np.sum((E_stor - E_freq_prime)**2)
ss_tot = np.sum((E_stor - np.mean(E_stor))**2)
R2_freq = 1 - ss_res / ss_tot
print(f"GMM from frequency: R2(E') = {R2_freq:.6f} ({gmm_freq._n_modes} active modes)")

gc.collect()
jax.clear_caches()

INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=11 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=1.297074s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=43 | final_cost=5.1732e+19 | elapsed=1.297s | final_gradient_norm=4.1638e+16


GMM from frequency: R2(E') = 0.934622 (5 active modes)


## 3. Predict E(t) from Frequency-Fitted Prony

If the Prony series is a true material property, it should predict E(t) without any refitting.

In [4]:
# Cross-predict: frequency Prony -> E(t)
E_t_from_freq = gmm_freq.predict(t, test_mode='relaxation')

ss_res_cross1 = np.sum((E_t - E_t_from_freq)**2)
ss_tot_cross1 = np.sum((E_t - np.mean(E_t))**2)
R2_cross1 = 1 - ss_res_cross1 / ss_tot_cross1

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(t, E_t / 1e6, 'go', ms=3, alpha=0.4, label='E(t) measured')
ax.loglog(t, E_t_from_freq / 1e6, 'r-', lw=2, label=f'E(t) from freq Prony (R2={R2_cross1:.4f})')
ax.set_xlabel('Time (s)')
ax.set_ylabel('E(t) (MPa)')
ax.set_title('Cross-Domain: Frequency Prony -> Relaxation Prediction')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.close('all')

print(f'Cross-domain R2 (freq -> relax) = {R2_cross1:.4f}')

Cross-domain R2 (freq -> relax) = -1.8348


## 4. Fit GMM to Relaxation Domain

In [5]:
# Fit to E(t)
gmm_time = GeneralizedMaxwell(n_modes=n_modes, modulus_type='tensile')

gmm_time.fit(t, E_t, test_mode='relaxation', optimization_factor=None)

E_t_fit = gmm_time.predict(t, test_mode='relaxation')

ss_res = np.sum((E_t - E_t_fit)**2)
ss_tot = np.sum((E_t - np.mean(E_t))**2)
R2_time = 1 - ss_res / ss_tot
print(f'GMM from relaxation: R2(E(t)) = {R2_time:.6f} ({gmm_time._n_modes} active modes)')

gc.collect()
jax.clear_caches()

INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=11 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=1.249876s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=11 | final_cost=4.7268e+19 | elapsed=1.250s | final_gradient_norm=2.6377e+20


GMM from relaxation: R2(E(t)) = 0.435170 (5 active modes)


## 5. Predict E*(omega) from Relaxation-Fitted Prony

In [6]:
# Cross-predict: relaxation Prony -> E*(omega)
E_star_from_time = gmm_time.predict(omega, test_mode='oscillation')
E_prime_from_time = E_star_from_time[:, 0]
E_double_from_time = E_star_from_time[:, 1]

ss_res_cross2 = np.sum((E_stor - E_prime_from_time)**2)
ss_tot_cross2 = np.sum((E_stor - np.mean(E_stor))**2)
R2_cross2 = 1 - ss_res_cross2 / ss_tot_cross2

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.loglog(omega, E_stor, 'ro', ms=3, alpha=0.4, label="E' measured")
ax1.loglog(omega, E_prime_from_time, 'r-', lw=2, label=f"E' from relax Prony")
ax1.set_xlabel(chr(969) + ' (rad/s)')
ax1.set_ylabel("E' (Pa)")
ax1.set_title(f"E' from Relaxation Prony (R2={R2_cross2:.4f})")
ax1.legend()

ax2.loglog(omega, E_loss, 'bs', ms=3, alpha=0.4, label='E" measured')
ax2.loglog(omega, E_double_from_time, 'b-', lw=2, label='E" from relax Prony')
ax2.set_xlabel(chr(969) + ' (rad/s)')
ax2.set_ylabel('E" (Pa)')
ax2.set_title('E" from Relaxation Prony')
ax2.legend()

plt.tight_layout()
plt.close('all')

print(f"Cross-domain R2 (relax -> freq) = {R2_cross2:.4f}")

Cross-domain R2 (relax -> freq) = 0.1200


## 6. Relaxation Spectrum Comparison

The discrete relaxation spectrum H(tau) should be consistent regardless of which domain was used for fitting.

In [7]:
# Extract Prony terms from both fits
prefix = 'E' if gmm_freq._modulus_type == 'tensile' else 'G'

def extract_prony(gmm):
    taus, Es = [], []
    for k in range(gmm._n_modes):
        E_k = gmm.parameters.get_value(f'{prefix}_{k+1}')
        tau_k = gmm.parameters.get_value(f'tau_{k+1}')
        if E_k > 1e-6:
            taus.append(tau_k)
            Es.append(E_k / 1e6)  # Pa -> MPa
    return np.array(taus), np.array(Es)

tau_freq, E_freq_modes = extract_prony(gmm_freq)
tau_time, E_time_modes = extract_prony(gmm_time)

# Load reference
df_ref = pd.read_csv(os.path.join(data_dir, 'prony_terms_reference.csv'), skiprows=[1])
df_ref.columns = df_ref.columns.str.strip()
tau_ref = df_ref['tau_i'].values
E_ref = df_ref['E_i'].values

fig, ax = plt.subplots(figsize=(12, 5))
w = 0.25
if len(tau_ref) > 0:
    ax.bar(np.log10(tau_ref) - w, E_ref, width=w, alpha=0.5, label=f'Reference ({len(tau_ref)} modes)', color='steelblue')
if len(tau_freq) > 0:
    ax.bar(np.log10(tau_freq), E_freq_modes, width=w, alpha=0.7, label=f'From freq ({len(tau_freq)} modes)', color='coral')
if len(tau_time) > 0:
    ax.bar(np.log10(tau_time) + w, E_time_modes, width=w, alpha=0.7, label=f'From relax ({len(tau_time)} modes)', color='green')
ax.set_xlabel('log10(tau / s)')
ax.set_ylabel('E_k (MPa)')
ax.set_title('Discrete Relaxation Spectrum: Domain Comparison')
ax.legend()
plt.tight_layout()
plt.close('all')

## 7. Consistency Summary

> **Note on FAST_MODE:** With only 5 Prony modes spanning 30+ decades of data,
> cross-domain predictions will be poor (negative R² is expected). For meaningful
> cross-domain validation, run with `FAST_MODE=0` and `n_modes >= 15`.
> The reference pyvisco data uses 30 Prony modes.

In [8]:
print('Cross-Domain Consistency Analysis')
print('=' * 50)
print(f"\nDirect fits:")
print(f"  Frequency domain R2(E') = {R2_freq:.4f}")
print(f"  Time domain R2(E(t))    = {R2_time:.4f}")
print(f"\nCross-predictions:")
print(f"  Freq Prony -> E(t):  R2 = {R2_cross1:.4f}")
print(f"  Time Prony -> E'(w): R2 = {R2_cross2:.4f}")
print(f"\nProny terms (non-negligible):")
print(f"  From frequency: {len(tau_freq)} modes")
print(f"  From relaxation: {len(tau_time)} modes")
print(f"  Reference: {len(tau_ref)} modes")
print(f"\nPhysical insight:")
print(f"  H(tau) is a domain-independent material property.")
print(f"  The relaxation spectrum should be consistent across")
print(f"  frequency and time domains. Deviations indicate")
print(f"  insufficient data coverage or model underfitting.")

Cross-Domain Consistency Analysis

Direct fits:
  Frequency domain R2(E') = 0.9346
  Time domain R2(E(t))    = 0.4352

Cross-predictions:
  Freq Prony -> E(t):  R2 = -1.8348
  Time Prony -> E'(w): R2 = 0.1200

Prony terms (non-negligible):
  From frequency: 5 modes
  From relaxation: 5 modes
  Reference: 30 modes

Physical insight:
  H(tau) is a domain-independent material property.
  The relaxation spectrum should be consistent across
  frequency and time domains. Deviations indicate
  insufficient data coverage or model underfitting.


## Key Takeaways

- **H(tau) is domain-independent**: The same relaxation spectrum describes E*(omega) and E(t)
- **Cross-domain validation** is a powerful consistency check for viscoelastic characterization
- **Prony series from either domain** should predict the other accurately
- **Deviations** in cross-predictions reveal data gaps or model limitations
- **Combined fitting** (both domains simultaneously) would give the most robust spectrum

## Related Notebooks

- `02_dmta_master_curve.ipynb`: Building master curves from multi-temperature data
- `04_dmta_relaxation.ipynb`: Time-domain relaxation fitting
- `07_dmta_tts_pipeline.ipynb`: Full TTS pipeline
- **Mode count matters**: Cross-domain R² requires `n_modes >= 15` for 30-decade data; 5 modes suffice only for single-domain fits


In [9]:
del gmm_freq, gmm_time
gc.collect()
jax.clear_caches()
print('Cleanup complete')

Cleanup complete
